Sacado de:
https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization

In [ ]:
#AGARRAR UNA SOLA IMAGEN CONTENIDO Y SE PUEDEN SELECCIONAR UNA O MAS IMAGENES ESTILO. LUEGO SE DESCARGAN SOLAS LAS IMAGENES
#EL TIEMPO QUE TARDA ES EN FUNCION DEL TAMAÑO DE IMAGEN QUE SE LE DA PARA COMPUTAR (SEA ESTILO O CONTENIDO)
#LA IMAGEN SALIDA VA A SER DEL MISMO TAMAÑO QUE LA DE ENTRADA (contenido) PERO CON UNA DEFINICION MENOR --> LETSENHANCE.IO

#PONER 1 AHORA SIEMPRE, DESPUES LO HARE PARA QUE AGARRE DIRECTO DEL DRIVE

#SE PUEDEN CARGAR HASTA 14 ESTILOS NOMAS // LIMITACIONES DEL GOOGLE COLAB

In [ ]:
#@title Settings
#@markdown >Alpha == Contrast /// Beta == Brightness // f(x) -> pixels originales // g(x) -> pixels after transf.l.

#@markdown >g(x) = α f(x) + β

#@markdown si a = 1 & b = 0 --> no se descarga la variante (solo original)

#@markdown >(0 sin collage, 1 con collage --> requiere 9 imagenes estilo)

collage = 0 #@param [0,1]

import tensorflow_hub as hub
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
from google.colab import files
import PIL.Image
from __future__ import print_function
from builtins import input
#import cv2 as cv #OJO ACA
import argparse

#drive.mount('/content/drive')
#!pip install spontit #SOLO HACE FALTA CORRER UNA VEZ ESTO
#from spontit import SpontitResource
#configuracion notificaciones al celular
#resource = SpontitResource("nicolas_raffa5828", "AWUC4HVUGE22LBE4Q4H1EACX6KBHG3NJCL0EYJFGBRNCMBK634JR52ZG0REXD7ZXABYUUPMY6QJNF4VY495P85D236QV69NOGY34")

ind = 0 #default en 0 > agarra la primer imagen contenido subida
lst = []

model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_image(img_path):
    img = tf.io.read_file(img_path) #leo archivo
    img = tf.image.decode_image(img , channels = 3) #me aseguro que tiene 3 canales (rgb)
    img = tf.image.convert_image_dtype(img , tf.float32) #me aseguro que tenga el formato correcto: float 32 bit
    img = img[tf.newaxis , :] #me aseguro que la imagen este en un array
    return img
    
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

source = input("Poner 1 si se va a subir desde la computadora, 0 si va a ser desde el drive:  ")

while not(source == "1" or source == "0")  :
  source = input ("Ingreso no valido, volver a escribir: ")
  print("Respuesta valida: " + source)

#OPCION SI SE SUBE DESDE LA COMPU
if source == "1":
  print("SELECCIONAR IMAGENES CONTENIDO")
  content_image = files.upload()

  for fn in content_image.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
     name=fn, length=len(content_image[fn])))
  
  contenidos = list(content_image.keys()) #LA IDEA IGUAL ES QUE SE HAGA DE A UNA SOLA IMAGEN CONTENIDO Y VARIAS DE ESTILO


  print("SELECCIONAR IMAGENES ESTILO")
  style_image = files.upload()
  
  for fn in style_image.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
     name=fn, length=len(style_image[fn])))
 
  estilos = list(style_image.keys())

content_image = load_image(contenidos[ind]) #no hace falta que este dentro del for porque uso siempre la misma de contenido

#modificar codigo para subir desde drive

for i in range(len(estilos)):
  
  print("CONTENT IMAGE")
  plt.imshow(np.squeeze(content_image)) #np.squeeze hace que se pueda visualizar el tensor
  plt.show()
  print(content_image.shape) #(1, 720, 1200 , 3) --> una imagen, height and width , channels
  
  style_image = load_image(estilos[i])
  print("STYLE IMAGE " + str(i))
  plt.imshow(np.squeeze(style_image))
  plt.show()  
  
  #aplico Neural Style Transfer

  stylized_image = model(tf.constant(content_image) , tf.constant(style_image))[0]
  
  new_image = tensor_to_image(stylized_image)
  

  #new_image = np.zeros(np.squeeze(stylized_image).shape, np.squeeze(stylized_image).dtype)

  alpha = 1.0 # Simple contrast control
  beta = 0    # Simple brightness control

  
  # Initialize values
  print(' Basic Linear Transforms ')
  print('-------------------------')
  try:
      alpha = 1 #@param {type:"slider", min:1, max:3, step:0.1} 
      #float(input('* Enter the alpha value [1.0-3.0]: ')) 
      beta = 0 #@param {type:"slider", min:0, max:100, step:5} 
      #int(input('* Enter the beta value [0-100]: '))
  except ValueError:
      print('Error, not a number')
  # Do the operation new_image(i,j) = alpha*image(i,j) + beta
  # Instead of these 'for' loops we could have used simply:
  new_image = cv2.convertScaleAbs(np.float32(new_image), alpha=alpha, beta=beta)
  # but we wanted to show you how to access the pixels :)
  #Alpha // contrast and Beta // brightness
  #for y in range(new_image.shape[0]):
    #  for x in range(new_image.shape[1]):
         # for c in range(new_image.shape[2]):
              #new_image[y,x,c] = np.clip(alpha*new_image[y,x,c] + beta, 0, 255)
  #print('Original Image')
  #cv2_imshow(stylized_image)
  # Wait until user press some key
  #cv.waitKey()


  print("STYLIZED IMAGE")
  plt.imshow(np.squeeze(stylized_image))
  plt.show()

  print('NEW IMAGE')
  plt.imshow(np.squeeze(new_image))
  plt.show()  

  file_name = contenidos[ind] + ' by ' + estilos[i] + '.jpg'
  tensor_to_image(stylized_image).save(file_name)

  file_name_cv = contenidos[ind] + ' by ' + estilos[i] + ' MODIFICADA.jpg'
  
  
  matplotlib.image.imsave(file_name_cv, new_image)
  
  

  #lst.append(file_name)
  #print(lst)

  try:
    from google.colab import files
  except ImportError:
    pass
  else:
    files.download(file_name)
    if alpha != 1.0 or beta != 0:
      files.download(file_name_cv)
  #response_true = resource.push("Imagen descargandose!", push_title = "Malerei Cuadros")#, expirationStamp = 5 )

#response_true = resource.push("Finalizo programa", push_title = "Malerei Cuadros")#, expirationStamp = 5 )


if collage == 1:

  from PIL import Image, ImageDraw
  import PIL
  import os
  import os.path
  from PIL import Image

  collage = Image.new("RGBA", (1500,1500), color=(255,255,255,255)) #1500 x 1500

  #f = r'C:/Users/nicor/OneDrive/Documentos/Python/Neural Style Transfer/collage test'
  #lst = os.listdir(f)

  c=0
  for i in range(0,1500,500):
      for j in range(0,1500,500):
          file = str(lst[c])
          photo = Image.open(file) #Image.open(file).convert("RGBA")
          photo = photo.resize((500,500))    #500 x 500    
          
          collage.paste(photo, (i,j))
          c+=1

  plt.imshow(np.squeeze(collage))
  plt.show() 
  file_name_collage = contenidos[ind] + '_collage.png'
  collage.save(file_name_collage)
  files.download(file_name_collage)
  #response_true = resource.push("Collage listo!", push_title = "Malerei Cuadros")#, expirationStamp = 5 )

  



Output hidden; open in https://colab.research.google.com to view.